In [1]:
import pywt, json, joblib
import cv2
import numpy as np
import base64
from sklearn.pipeline import Pipeline

In [2]:
face_cascade = cv2.CascadeClassifier("./opencv/haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("./opencv/haarcascades/haarcascade_eye.xml")

In [3]:
def get_cropped_image_for_2_eyes(image_path):
    img_main =cv2.imread(image_path) 
    if img_main is None:
        print('Not an image')
        return None
    else:
        try:
            gray_img = cv2.cvtColor(img_main, cv2.COLOR_BGR2GRAY)
        except:
            print('Gray image is not available')
            return None
        else:
            faces_gray = face_cascade.detectMultiScale(gray_img, 1.1, 5)
            for (x,y,w,h) in faces_gray:
                roi_color = img_main[y:y+h, x:x+w]
                eyes_gray = eye_cascade.detectMultiScale(gray_img, 1.1, 5)
                if len(eyes_gray) >=2:
                    cv2.imwrite('./uploads/cropped.png', roi_color)

In [4]:
def w2d(img, mode='db1', level=5):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255
    imArray_H =  np.uint8(imArray_H)

    return imArray_H


def get_the_features(image):
    X = []
    img = cv2.imread(image) 
    if img is not None:  
        scalled_img = cv2.resize(img, (32, 32)) 
        wavelet_img = w2d(img, 'db1', 5 )
        scalled_wavelet_img = cv2.resize(wavelet_img, (32, 32)) 

        combined_image = np.vstack((scalled_img.reshape(32*32*3,1), scalled_wavelet_img.reshape(32*32,1))) # I'm stacking 
        X.append(combined_image)
        return X

In [7]:
get_cropped_image_for_2_eyes('./uploads/cristiano-ronaldo-al-nassr-2023-1692731063-114594.jpg')

In [8]:
X = get_the_features('./uploads/cropped.png')

In [9]:
print(X)

[array([[149],
       [153],
       [128],
       ...,
       [ 21],
       [  1],
       [  0]], dtype=uint8)]


In [10]:
X = np.array(X).reshape(len(X),4096).astype('float')
X.shape

(1, 4096)

In [14]:
model = joblib.load('./artifacts/model.pkl')

In [28]:
print(model.predict(X)[0])

4


In [24]:
np.set_printoptions(suppress=True)

In [26]:
model.predict_proba(X)*100

array([[ 0.01344681,  0.0329523 ,  0.00742812,  0.01284548, 99.93332729]])

In [31]:
for i in model.predict_proba(X)[0]*100:
    print(i)

0.013446808633419176
0.032952299773476905
0.007428116974708331
0.012845481039459788
99.93332729357893
